This notebook intends to serve as the basis for LIneA doc TE 001

In [134]:
%matplotlib inline
import pandas, numpy
from scipy.linalg import block_diag

# Input data

CSV file to be easily loaded as a MultiIndex Pandas.DataFrame

Here's an example of how to load the data and a view of it:

In [31]:
cldf = pandas.read_csv("../dat/apsmeasurements_dl15lmin0lmax450nside1024.csv.gz", header=[0,1], index_col=0)
cldf.head(2)

zbin    zp0607                                                             \
data     mock0     mock1    mock2     mock3     mock4     mock5     mock6   
ell                                                                         
8.0   0.000014  0.000106  0.00006  0.000033  0.000116  0.000047  0.000034   
23.0  0.000079  0.000064  0.00005  0.000073  0.000078  0.000107  0.000109   

zbin                                  ...       zp0910                      \
data     mock7     mock8     mock9    ...     mock1791  mock1792  mock1793   
ell                                   ...                                    
8.0   0.000024  0.000048  0.000040    ...     0.000010  0.000029  0.000026   
23.0  0.000115  0.000054  0.000054    ...     0.000053  0.000020  0.000021   

zbin                                                                        
data  mock1794  mock1795  mock1796  mock1797  mock1798  mock1799        y1  
ell                                                                         
8.0   0.000024  0.000030  0.000010  0.000018  0.000018  0.000011  0.000060  
23.0  0.000030  0.000031  0.000024  0.000036  0.000025  0.000033  0.000024  

[2 rows x 7204 columns]